In [1]:
pow(2,100*100)

1995063116880758384883742162683585083823496831886192454852008949852943883022194663191996168403619459789933112942320912427155649134941378111759378593209632395785573004679379452676524655126605989552055008691819331154250860846061810468550907486608962488809048989483800925394163325785062156830947390255691238806522509664387444104675987162698545322286853816169431577562964076283688076073222853509164147618395638145896946389941084096053626782106462142733339403652556564953060314268023496940033593431665145929777327966577560617258203140799419817960737824568376228003730288548725190083446458145465055792960141483392161573458813925709537976911927780082695773567444412306201875783632550272832378927071037380286639303142813324140162419567169057406141965434232463880124885614730520743199225961179625013099286024170834080760593232016126849228849625584131284406153673895148711425631511108974551420331382020293164095759646475601040584584156607204496286701651506192063100418642227590867090057460641785695191145605506

In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt

In [3]:
env=gym.make('FrozenLake-v0')

In [5]:
Q=np.zeros([env.observation_space.n, env.action_space.n])

In [6]:
learning_rate=.85
dis=.99
num_episodes=2000

In [9]:
rList=[]
for i in range(num_episodes):
    state=env.reset()
    done=False
    rAll=0
    while not done:
        action=np.argmax(Q[state,:]+np.random.randn(1, env.action_space.n)/(i+1))  #상/하/좌/우
        new_state, reward, done, _ =env.step(action)
        #q_table 갱신
        Q[state, action]=reward+dis*np.max(Q[new_state, :])
        state=new_state
        rAll+=reward
    rList.append(rAll)
print("score : "+str(sum(rList)/num_episodes))
print(Q)
        
        

score : 0.0175
[[0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.99 0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   0.  ]]


In [10]:
#q-learning + nn

In [11]:
import tensorflow as tf

In [13]:
env=gym.make('FrozenLake-v0')

In [15]:
input_size=env.observation_space.n
output_size=env.action_space.n
learning_rate=0.1

In [16]:
X=tf.placeholder(shape=[1,input_size],dtype=tf.float32) #(1,16)
W=tf.Variable(tf.random_uniform([input_size,output_size],0,0.01))

Qpred=tf.matmul(X,W) #(1,4)
Y=tf.placeholder(shape=[1,output_size], dtype=tf.float32)

loss=tf.reduce_sum(tf.square(Y-Qpred))
train=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [17]:
dis=.99
num_episodes=2000

rList=[]

In [29]:
def one_hot(x):
    return np.identity(16)[x:x+1]
#one_hot(3)

In [33]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        s=env.reset() # s=0
        e=1. / ((i/50)+10) #e-greedy 0.1 -> 0.102 -> .... 0근사
        rAll=0
        done=False
    
        while not done:
            Qs=sess.run(Qpred, feed_dict={X:one_hot(s)})
            #exploit & exploration
            if np.random.rand(1) < e : #10%확률로 탐험
                a=env.action_space.sample()
            else :
                a=np.argmax(Qs)

            s1, reward, done, _ =env.step(a)    

            if done: #terminal state임
                Qs[0,a]=reward            
            else:#terminal state가 아님
                Qs1=sess.run(Qpred,feed_dict={X:one_hot(s1)})
                Qs[0,a]=reward+dis*np.max(Qs1)
            sess.run(train, feed_dict={X:one_hot(s), Y:Qs})

            rAll+=reward
            s=s1
        rList.append(rAll)
print(str(sum(rList)/num_episodes)+"%")
            
            
            
    

0.4725%


In [25]:
#np.identity(16)
#np.identity(4)[0:0+1]